In [1]:
import praw
import pprint as pp
import os
import json

from classes.imgdir_handler import imgdir_handler

In [2]:
handler = imgdir_handler()

In [3]:
handler.set_table('dankmemes_2019_1')

In [4]:
imgdir_gen = handler.imgdir_generator()

In [5]:
next(imgdir_gen)

100%|██████████| 101/101 [00:25<00:00,  4.04it/s]


In [6]:
next(imgdir_gen)

100%|██████████| 101/101 [00:24<00:00,  4.19it/s]
